# Special class methods

In [ ]:
import random

class Vector:        
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) != int or type(y) != int:
            raise AttributeError('x and y should be int')
        
        self._x = x
        self._y = y
        self._color = color
    
    def get_x(self):
        return self._x
    
    def get_y(self):
        return self._y

Methods with double underscore (dunder) at the beginning and the end of their names have special meaning.

We are already familiar with `__next__` and `__iter__`, it's time to learn about the rest.

In [ ]:
vector = Vector(1, 2, 'red')
str(vector)

In [ ]:
class VectorWithStr(Vector):
    def __str__(self):
        return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)

In [7]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

initializing a vector


'vector (1, 2) of color red'

__Q:__ Casting to string and that's all?

__A:__ Of course not. Implicit conversions sometimes occur where we do not expect them

In [6]:
print(vector)

vector (1, 2) of color red


In [8]:
print("OBJECT: {}".format(vector))

OBJECT: vector (1, 2) of color red


In [9]:
mylist = [vector]
print(mylist)

__Q:__ And where are the ugly lines again from?!

__A:__ Python uses two methods of casting to a string. These are the functions `str` and` repr`, which differ in their purpose.

`str` is used where human readability is needed, and` repr` is implemented so that it is possible to unambiguously determine which object we are talking about. If `repr` is not implemented, the standard option is used, and if` str` is not implemented, then `repr` is used instead.

Let's try!

In [10]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [11]:
vector = VectorWithRepr(1, 2, 'red')

print(vector)
mylist = [vector]
print(mylist)
mydict = {}
mydict[vector]

initializing a vector
vector representation (x: 1, y: 2, color: red)
[vector representation (x: 1, y: 2, color: red)]


KeyError: ignored

In [12]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [13]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
# now we should get different outputs
print(vector)
print([vector])

initializing a vector
vector (1, 2) of color red
[vector representation (x: 1, y: 2, color: red)]


## Arithmetic

In [14]:
import math
import random

class VectorWithMath(VectorWithBothReprAndStr):    
    def __abs__(self):
        return math.hypot(self._x, self._y)
    
    def __add__(self, other):
        return VectorWithMath(self.get_x() + other.get_x(),
                     self.get_y() + other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    def __sub__(self, other):
        return VectorWithMath(self.get_x() - other.get_x(),
                     self.get_y() - other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    # there also div, mul and many other methods

In [15]:
vector1 = VectorWithMath(3, 4, 'blue')
vector2 = VectorWithMath(1, 2, 'red')

initializing a vector
initializing a vector


In [16]:
print(abs(vector1))
print(vector1 + vector2)

5.0
initializing a vector
vector (4, 6) of color red


## Type conversions

In [17]:
import math

class VectorWithTypes(VectorWithMath):
    def __bool__(self):
        return bool(self._x) or bool(self._y)
    
    def __int__(self): # int()
        return int(float(self))
    
    def __float__(self):
        return abs(self)

In [18]:
vector = VectorWithTypes(3, 4, 'blue')
print(vector)
print(int(vector))
print(float(vector))
if vector:
    print("vector ~ True")

initializing a vector
vector (3, 4) of color blue
5
5.0
vector ~ True


In [19]:
vector = VectorWithTypes()
print(vector)
if not vector:
    print("vector ~ False")

initializing a vector
vector (0, 0) of color None
vector ~ False


## Iterating

We already know one way to make the object "iterable", the `__iter__` and `__next__` methods. But it is not the only way.

In [20]:
class VectorIterable(VectorWithTypes):
    def __getitem__(self, position):
        return (self._x, self._y)[position]
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [21]:
vector = VectorIterable(100, 500)
print(vector[0])
print(vector[3])

initializing a vector
100


IndexError: ignored

In [22]:
for coordinate in vector:
    print(coordinate)

100
500


In [23]:
for coordinate in reversed(vector):
    print(coordinate)

500
100


In [24]:
class VectorIterable(VectorWithTypes):
    
    def __getitem__(self, position):
        return (self._y, self._x)[position]
    
    def __iter__(self):
        
        #return iter((self._x, self._y))
        return self
    
    def __next__(self):
        
        if not hasattr(self, 'count'):
            self.count = 0
        
        self.count += 1
        
        if self.count < 3:
            return (self._x, self._y)[self.count - 1]
        else:
            raise StopIteration
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [25]:
vect = VectorIterable(3, 5)

for c in vect:
    print(c)

initializing a vector
3
5


## Dynamic work with attributes

It seems that in python there is no protection against "hacking". But is it possible to do it yourself?

In [174]:
class VectorWithAllAttributes(VectorIterable):
    def __getattr__(self, attr_name):
        return "value of {}".format(attr_name)
    
    def __setattr__(self, attr_name, attr_value):
        if attr_name not in ('_x', '_y', '_color'):
            raise Exception('you shall not add new attributes here, young padawan!')
        else:
            super().__setattr__(attr_name, attr_value)
            
    def __delattr__(self, attr_name):
        print('Heh, you can delete nothing')

In [175]:
vector = VectorWithAllAttributes(1, 2, 'violet')
print(dir(vector))

initializing a vector
['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__float__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__int__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_color', '_x', '_y', 'get_x', 'get_y']


In [176]:
print(vector.some_attribute)
print(vector._color)
print(vector.get_x())

value of some_attribute
violet
1


In [177]:
vector.new_attribute = "value"

Exception: ignored

In [178]:
del vector._color
delattr(vector, '_color')
print(vector._color)

Heh, you can delete nothing
Heh, you can delete nothing
violet


### ```__getattr__ vs. __getattribute__```

In [179]:
class GetAttr:
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattr__(self, attr):   # for non-existing attrs only
        print('get: ' + attr)      # Not for attr1: it's inherited from the class
        return 3                   # Not for attr2: it exists for the object
    

class GetAttribute:
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattribute__(self, attr):  # Is called by all assigning operations
        print('get: ' + attr)          # Use superclass in order to not get stuck in an infinite cycle
        if attr == 'attr3':
            return 3
        else:
            return super().__getattribute__(attr)

In [180]:
X = GetAttr()
print(X.attr1)
print(X.attr2)
print(X.attr3)
print('-' * 40)        
X = GetAttribute()
print(X.attr1)
print(X.attr2)
print(X.attr3)

1
2
get: attr3
3
----------------------------------------
get: attr1
1
get: attr2
2
get: attr3
3


## Contexts

In [217]:
class VectorWithContextManager(VectorWithAllAttributes):
    def __enter__(self):
        print('entering context')
    def __exit__(self, *args):
        print(args)
        print(dir(args[2]), args[2].tb_lineno)
        print('leaving context')
        return False # -- throwing error 
        # return True  # -- not throwing error

In [182]:
try:
    with VectorWithContextManager() as vec:
        for i in range(3):
            print(i)
        raise Exception('something happened inside!')
except Exception:
    print('an exception was raised...')
    pass
print('we are out of the context')

initializing a vector
entering context
0
1
2
(<class 'Exception'>, Exception('something happened inside!'), <traceback object at 0x7effa68030f0>)
['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next'] 5
leaving context
we are out of the context


But we can do better!

In [183]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield Vector()
    print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


In [184]:
# And now with error handling:

@contextmanager
def vector_mgr():
    try:
        print('handling entering the context')
        yield Vector()
    except ZeroDivisionError as e:
        pass
    finally:
        print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
        infin = 1 / 0
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


## Creating and deleting objects

In [218]:
class VectorInitialized(VectorWithContextManager):
    def __new__(cls, *args, **kwargs):
        print('invoking __new__ method')
        print(cls, args, kwargs)
        print(object)
        return object.__new__(cls)
    
    def __del__(self):
        print('deleting an object')
        raise Exception("exception while destructing")

In [224]:
vector = VectorInitialized(1, 2, color='navy blue')
print(vector)

invoking __new__ method
<class '__main__.VectorInitialized'> (1, 2) {'color': 'navy blue'}
<class 'object'>
initializing a vector
vector (1, 2) of color navy blue


In [225]:
del vector

Exception ignored in: <function VectorInitialized.__del__ at 0x7effa67fe560>
Traceback (most recent call last):
  File "<ipython-input-218-0767888d90a6>", line 10, in __del__
Exception: exception while destructing


deleting an object


### Task! 

How to use the `__new__` method in order to create a singleton class? i.e. the class allowing to create the only object and returning the same object when you try to create another one.

In [165]:
class SingletonClass:
    
    #...your code here...
    
    def __new__(cls, *args, **kwargs):
        #...your code here...
        

obj1 = SingletonClass()
obj2 = SingletonClass()
assert id(obj1) == id(obj2)

### Function annotations - https://www.python.org/dev/peps/pep-3107/

In [89]:
def Sum(x: 10, y: 'term 2', z: int = 0) -> max(2, 9):
    "returns x + y + z"
    return x + y + z
help(Sum)

Sum('for ','reading ','only')

Help on function Sum in module __main__:

Sum(x: 10, y: 'term 2', z: int = 0) -> 9
    returns x + y + z



'for reading only'

## Once again on modules - https://docs.python.org/3/tutorial/modules.html